In [1]:
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import MessagesState,add_messages
from langgraph.prebuilt import create_react_agent,InjectedState
from langgraph_supervisor import create_supervisor,create_handoff_tool
from langgraph_swarm import create_swarm
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict,Annotated
from langgraph.types import Command,Interrupt
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
import os
load_dotenv()

True

In [2]:
memry=InMemorySaver()

In [3]:
def search(query:str):
    '''the search tool'''
    tool=TavilySearchResults(max_results=2)
    tool.invoke(query)

@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

In [4]:
model=ChatGoogleGenerativeAI(api_key=os.getenv('google'),model='gemini-2.5-flash')

In [8]:
math_agent=create_react_agent(
    model=model,
    tools=[add,multiply,transfer_to_search_agent],
    name='math_agent',
    prompt='you are a mathematician'
)

search_agent =create_react_agent(
    model=model,
    tools=[search,transfer_to_math_agent],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

In [9]:
swarm=create_swarm(
    agents=[math_agent,search_agent],
    default_active_agent='research_expert'
)

In [10]:
swarm=swarm.compile(
)

In [14]:
workflow=swarm.invoke(
      {
        "messages": [
            {
                "role": "user",
                "content": "what is 8 plus 9"
            }
        ]
    }
)


In [15]:
for i in workflow['messages']:
    i.pretty_print()

================================ Human Message =================================

what is 8 plus 9
================================== Ai Message ==================================
Name: research_expert
Tool Calls:
  transfer_to_math_agent (0a03c47c-15b4-47af-b7b0-a80421311192)
 Call ID: 0a03c47c-15b4-47af-b7b0-a80421311192
  Args:
================================= Tool Message =================================
Name: transfer_to_math_agent

Successfully transferred to math_agent
================================== Ai Message ==================================
Name: math_agent
Tool Calls:
  add (4739db6b-7c12-418d-b2ee-a9aee9eb699e)
 Call ID: 4739db6b-7c12-418d-b2ee-a9aee9eb699e
  Args:
    b: 9.0
    a: 8.0
================================= Tool Message =================================
Name: add

17.0
================================== Ai Message ==================================
Name: math_agent

8 plus 9 is 17.


In [ ]:
workflow=create_supervisor(
    agents=[math_agent,search_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )

)

In [ ]:
app=workflow.compile(checkpointer=memry)

In [ ]:
config={'configurable':{'thread_id':'1'}}

In [ ]:
result=app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is quantum computing?"
        }
    ]},config=config
)

/var/folders/kh/4f4bk94j1d36fl9w_7g2nhdm0000gn/T/ipykernel_30210/3361027083.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool=TavilySearchResults(max_results=2)
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


In [ ]:
for r in result['messages']:
    r.pretty_print()

================================ Human Message =================================

what is quantum computing?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (82638533-1658-4f35-837e-f27404b1221a)
 Call ID: 82638533-1658-4f35-837e-f27404b1221a
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

Quantum computing is a new type of computing that uses the principles of quantum mechanics to solve complex problems that are beyond the capabilities of classical computers. Unlike classical computers that store information as bits (0s or 1s), quantum computers use "qubits" which can represent 0, 1, or both simultaneously through a phenomenon called superposition. This allows them to proces

In [ ]:
result

{'messages': [HumanMessage(content='what is quantum computing?', additional_kwargs={}, response_metadata={}, id='f37170f7-c5ef-4d97-aa8d-73ec43274912'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'transfer_to_research_expert', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, name='supervisor', id='run--c49a4e29-d5e0-4f4d-aba6-bba59335b22f-0', tool_calls=[{'name': 'transfer_to_research_expert', 'args': {}, 'id': '82638533-1658-4f35-837e-f27404b1221a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 106, 'output_tokens': 59, 'total_tokens': 165, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 45}}),
  ToolMessage(content='Successfully transferred to research_expert', name='transfer_to_research_expert', id='81a3df61-4b23-46a8-b395-1c733895bca9', tool_call_id='82638533-1658-4f35-837e-f27404b12

In [ ]:
transfer_to_math_agent=create_handoff_tool(
    agent_name='math_agent',
    description='handoff to math agent for math problems',
)

transfer_to_search_agent=create_handoff_tool(
    agent_name='research_expert',
    description='handoff to search agent for current events'
)